# Wan 2.2 Worker Node (Kaggle Edition)

Run this notebook on Kaggle (GPU T4 x2 or P100) to join the distributed pool.

**Features:**
- **Auto-Scaling:** Automatically launches 2 workers if 2 GPUs are detected (for 1.3B model).
- **Safety:** Limits to 1 worker for the 14B model to prevent System RAM OOM.

**Pre-requisites:**
1.  You must have the Controller running (e.g., on a VPS or via Ngrok).
2.  Copy your Controller URL (e.g., `wss://my-controller.ngrok.io/ws/worker`).

In [ ]:
# 1. Install Dependencies
!pip install -q diffusers transformers accelerate sentencepiece websockets pydantic fastapi uvicorn

In [ ]:
# 2. Clone the Codebase
!git clone https://github.com/your-repo/wan_pool.git
%cd wan_pool

In [ ]:
# 3. Configure & Run Worker(s)
import os
import torch
import subprocess
import time
import signal
import sys

# --- CONFIGURATION ---
CONTROLLER_URL = "wss://your-controller-url.com/ws/worker"  # <--- PASTE YOUR URL HERE
MODEL_ID = "Wan-AI/Wan2.1-T2V-1.3B-Diffusers" # Or "Wan-AI/Wan2.1-T2V-14B-Diffusers"
# ---------------------

def launch_workers():
    gpu_count = torch.cuda.device_count()
    print(f"Detected {gpu_count} GPUs.")
    
    # Decision Logic
    num_workers = gpu_count
    if "14B" in MODEL_ID and gpu_count > 1:
        print("WARNING: 14B Model detected. Restricting to 1 Worker to prevent System RAM OOM.")
        num_workers = 1
    
    processes = []
    try:
        for i in range(num_workers):
            print(f"Launching Worker {i+1}/{num_workers} on GPU {i}...")
            env = os.environ.copy()
            env["CONTROLLER_URL"] = CONTROLLER_URL
            env["MODEL_ID"] = MODEL_ID
            env["CUDA_VISIBLE_DEVICES"] = str(i)
            env["WORKER_NAME"] = f"Kaggle-Worker-GPU{i}"
            
            # Start process
            p = subprocess.Popen(
                [sys.executable, "-m", "src.worker_client"],
                env=env
            )
            processes.append(p)
        
        print(f"Running {len(processes)} workers. Press Stop to exit.")
        # Wait for processes
        for p in processes:
            p.wait()
            
    except KeyboardInterrupt:
        print("Stopping workers...")
        for p in processes:
            p.terminate()

if __name__ == "__main__":
    launch_workers()